https://nbviewer.jupyter.org/github/microsoft/recommenders/blob/master/notebooks/00_quick_start/sar_movielens.ipynb

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# from datetime import datetime

# def convert_date(str_date):
#   return datetime.strptime(str_date, "%Y-%m-%d").date()

# df['stayDate'] = df['tripInfo.stayDate'].apply(convert_date)
# print(df.info())


# users = df.groupby('userId')['userId'].agg([('users','count')]).reset_index().reset_index().iloc[:,:2]
# new_df = pd.merge(users,df, on='userId').rename(columns={'index':'userID'})
# new_df.head()

In [5]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', 25)

df = pd.read_csv('/content/gdrive/My Drive/tobigs_recommend/Phase 1. Data Collection & Preprocessing/1. Data/Hotel_Data/TA_User_Reviws_Korea_all_new_df.csv')
df.head()

,userID,rating,location.locationId,location.placeType,location.reviewSummary.count,location.reviewSummary.rating,location.reviewSummary.locationId,userProfile.hometown.location.name,stayDate
0,0,4,306130,ACCOMMODATION,4033.0,4.5,306130.0,Jakarta,2016-02-29
1,0,2,6352819,ATTRACTION,276.0,4.5,6352819.0,Jakarta,2016-02-29
2,1,5,9033360,EATERY,10.0,4.5,9033360.0,Seoul,2015-08-31
3,1,5,9017499,EATERY,8.0,4.0,9017499.0,Seoul,2015-11-30
4,1,5,4076062,EATERY,7.0,4.0,4076062.0,Seoul,2015-06-30


In [6]:
COLUMNS = ['userID', 'location.locationId', 'rating', 'stayDate']
df = df.loc[:,COLUMNS]
df.head()

,userID,location.locationId,rating,stayDate
0,0,306130,4,2016-02-29
1,0,6352819,2,2016-02-29
2,1,9033360,5,2015-08-31
3,1,9017499,5,2015-11-30
4,1,4076062,5,2015-06-30


In [0]:
! pip install papermill

In [17]:
cd /content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion

/content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion


In [19]:
cd reco_utils

/content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion/reco_utils


In [20]:
from google.colab import files
files.upload()

{}

In [12]:
import itertools
import logging
import os
import gc

import numpy as np
import pandas as pd
import papermill as pm
from sklearn.externals import joblib 

# from python_splitters import python_stratified_split
from python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as sk_split

from constants import (
    DEFAULT_ITEM_COL,
    DEFAULT_USER_COL,
    DEFAULT_TIMESTAMP_COL,
)
# from split_utils import (
#     process_split_ratio,
#     min_rating_filter_pandas,
#     split_pandas_data_with_ratios,
# )

import pandas as pd
import numpy as np
import math

from constants import DEFAULT_ITEM_COL, DEFAULT_USER_COL

try:
    from pyspark.sql.functions import col, broadcast
except ImportError:
    pass  # so the environment without spark doesn't break


def process_split_ratio(ratio):
    if isinstance(ratio, float):
        if ratio <= 0 or ratio >= 1:
            raise ValueError("Split ratio has to be between 0 and 1")

        multi = False
    elif isinstance(ratio, list):
        if any([x <= 0 for x in ratio]):
            raise ValueError(
                "All split ratios in the ratio list should be larger than 0."
            )

        # normalize split ratios if they are not summed to 1
        if math.fsum(ratio) != 1.0:
            ratio = [x / math.fsum(ratio) for x in ratio]

        multi = True
    else:
        raise TypeError("Split ratio should be either float or a list of floats.")

    return multi, ratio


def min_rating_filter_pandas(
    data,
    min_rating=1,
    filter_by="user",
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
):
    split_by_column, _ = _check_min_rating_filter(
        filter_by, min_rating, col_user, col_item
    )
    rating_filtered = data.groupby(split_by_column).filter(
        lambda x: len(x) >= min_rating
    )
    return rating_filtered


def min_rating_filter_spark(
    data,
    min_rating=1,
    filter_by="user",
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
):
    
    split_by_column, split_with_column = _check_min_rating_filter(
        filter_by, min_rating, col_user, col_item
    )
    rating_temp = (
        data.groupBy(split_by_column)
        .agg({split_with_column: "count"})
        .withColumnRenamed("count(" + split_with_column + ")", "n" + split_with_column)
        .where(col("n" + split_with_column) >= min_rating)
    )

    rating_filtered = data.join(broadcast(rating_temp), split_by_column).drop(
        "n" + split_with_column
    )
    return rating_filtered


def _check_min_rating_filter(filter_by, min_rating, col_user, col_item):
    if not (filter_by == "user" or filter_by == "item"):
        raise ValueError("filter_by should be either 'user' or 'item'.")

    if min_rating < 1:
        raise ValueError("min_rating should be integer and larger than or equal to 1.")

    split_by_column = col_user if filter_by == "user" else col_item
    split_with_column = col_item if filter_by == "user" else col_user
    return split_by_column, split_with_column


def split_pandas_data_with_ratios(data, ratios, seed=42, shuffle=False):
    
    if math.fsum(ratios) != 1.0:
        raise ValueError("The ratios have to sum to 1")

    split_index = np.cumsum(ratios).tolist()[:-1]

    if shuffle:
        data = data.sample(frac=1, random_state=seed)

    splits = np.split(data, [round(x * len(data)) for x in split_index])

    # Add split index (this makes splitting by group more efficient).
    for i in range(len(ratios)):
        splits[i]["split_index"] = i

    return splits


def _do_stratification(
    data,
    ratio=0.75,
    min_rating=1,
    filter_by="user",
    is_random=True,
    seed=42,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_timestamp=DEFAULT_TIMESTAMP_COL,
):
    # A few preliminary checks.
    if not (filter_by == "user" or filter_by == "item"):
        raise ValueError("filter_by should be either 'user' or 'item'.")

    if min_rating < 1:
        raise ValueError("min_rating should be integer and larger than or equal to 1.")

    if col_user not in data.columns:
        raise ValueError("Schema of data not valid. Missing User Col")

    if col_item not in data.columns:
        raise ValueError("Schema of data not valid. Missing Item Col")

    if not is_random:
        if col_timestamp not in data.columns:
            raise ValueError("Schema of data not valid. Missing Timestamp Col")

    multi_split, ratio = process_split_ratio(ratio)

    split_by_column = col_user if filter_by == "user" else col_item

    ratio = ratio if multi_split else [ratio, 1 - ratio]

    if min_rating > 1:
        data = min_rating_filter_pandas(
            data,
            min_rating=min_rating,
            filter_by=filter_by,
            col_user=col_user,
            col_item=col_item,
        )

    # Split by each group and aggregate splits together.
    splits = []

    # If it is for chronological splitting, the split will be performed in a random way.
    df_grouped = (
        data.sort_values(col_timestamp).groupby(split_by_column)
        if is_random is False
        else data.groupby(split_by_column)
    )

    for name, group in df_grouped:
        group_splits = split_pandas_data_with_ratios(
            df_grouped.get_group(name), ratio, shuffle=is_random, seed=seed
        )

        # Concatenate the list of split dataframes.
        concat_group_splits = pd.concat(group_splits)

        splits.append(concat_group_splits)

    # Concatenate splits for all the groups together.
    splits_all = pd.concat(splits)

    # Take split by split_index
    splits_list = [
        splits_all[splits_all["split_index"] == x].drop("split_index", axis=1)
        for x in range(len(ratio))
    ]

    return splits_list



def python_stratified_split(
    data,
    ratio=0.75,
    min_rating=1,
    filter_by="user",
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    seed=42,
):

    return _do_stratification(
        data,
        ratio=ratio,
        min_rating=min_rating,
        filter_by=filter_by,
        col_user=col_user,
        col_item=col_item,
        is_random=True,
        seed=seed,
    )

In [7]:
df.head()

,userID,location.locationId,rating,stayDate
0,0,306130,4,2016-02-29
1,0,6352819,2,2016-02-29
2,1,9033360,5,2015-08-31
3,1,9017499,5,2015-11-30
4,1,4076062,5,2015-06-30


In [15]:
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

ModuleNotFoundError: ignored

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, 0.75)
print(train.shape, test.shape)

TypeError: ignored

In [0]:
# Constants to be used
TOP_K = 10
USER, ITEM, RATING, TIME, PRED = 'userID', 'location.locationId', 'rating', 'stayDate', 'SCORE'  

In [115]:
data = new_df.loc[:,[USER,ITEM,RATING,TIME]]
data

,userID,location.locationId,rating,stayDate
0,0,306130,4,2016-02-29
1,0,6352819,2,2016-02-29
2,1,9033360,5,2015-08-31
3,1,9017499,5,2015-11-30
4,1,4076062,5,2015-06-30
...,...,...,...,...
28556,4555,4105582,5,2015-07-31
28557,4556,306126,2,2016-11-30
28558,4557,8692662,4,2019-03-31
28559,4558,7358712,4,2019-05-31


In [0]:
# 반드시 실행해야 함!
header = {
    "col_user": USER,
    "col_item": ITEM,
    "col_rating": RATING,
    "col_timestamp": TIME,
    "col_prediction": PRED,
}

In [0]:
# 이미 데이터가 나누어져 있으면 이 부분 Skip하고, 나누어진 데이터를 train과 test 이름의 변수에 저장!
train, test = python_stratified_split(data, ratio=0.75, col_user=header["col_user"], col_item=header["col_item"], seed=42)

In [118]:
print('train shape:', train.shape, 'test shape:', test.shape)

train shape: (22167, 4) test shape: (6394, 4)


In [119]:
# 이미 데이터가 나누어져 있으면 이 부분 Skip !
# 객체를 pickled binary file 형태로 저장한다 
joblib.dump([data, train, test], 'sar_data.pkl')

del data; gc.collect()

374

### Build SAR Model

In [0]:
from sar_singlenode import SARSingleNode

In [0]:
# 이 부분부터 아래 끝까지 실행해야 함!

# set log level to INFO
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)-8s %(message)s')

model = SARSingleNode(
    similarity_type='cooccurrence',   # 'jaccard', 'lift', 'cooccurrence' 
    time_decay_coefficient=30, 
    time_now=None, 
    timedecay_formula=True, 
#    normalize=True,
    **header
)

In [122]:
model.fit(train)
#joblib.dump(model, 'sar_model.pkl')

2020-05-26 11:13:07,559 INFO     Collecting user affinity matrix
2020-05-26 11:13:07,569 INFO     Calculating time-decayed affinities


TypeError: ignored

### Make Recommendations

In [0]:
top_k = model.recommend_k_items(test, remove_seen=True)
#joblib.dump(top_k, 'sar_top_k.pkl')
top_k.head(10)

### Evaluate SAR Model

In [0]:
args = [test, top_k]
kwargs = dict(col_user=USER, 
              col_item=ITEM, 
              col_rating=RATING, 
              col_prediction=PRED, 
              relevancy_method='top_k', 
              k=TOP_K)

#eval_map = map_at_k(*args, **kwargs)
#eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

print(f"Model:",
      f"Top K:\t\t {TOP_K}",
#      f"MAP:\t\t {eval_map:f}",
#      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')